In [2]:
# Basic data handling
import pandas as pd
import numpy as np


In [3]:
# Loading the hotel review dataset
df = pd.read_csv('Combined_TripAdvisor_Guest_Reviews_in_ Salalah_Oman_Dataset (1).csv')
df.head(10)

,Hotel Name,User Location,Rating,Review Title,Review Text,Helpful Votes,Trip Type,Stay Date,Created Date,Published Date,Language,Value,Rooms,Location,Cleanliness,Service,Sleep Quality
0,Al Baleed Resort Salalah by Anantara,NaN,5,Ein Erlebnis mit excellenten Mitarbeitern v Ho...,"Wir hatten das große Glück, eine Woche in dies...",0,FRIENDS,31/12/2024,27/12/2024,27/12/2024,German,5,5,5,5,5,5
1,Al Baleed Resort Salalah by Anantara,Paris,5,Three days in a paradise,At our very late arrival Veronica very nice an...,1,NONE,29/02/2024,21/02/2024,20/02/2024,English,5,5,5,5,5,5
2,Al Baleed Resort Salalah by Anantara,Lengnau,5,Wohlfühloase pur,Ein Hotel das keine Wünsche offen lässt. \nWun...,0,SOLO,31/12/2024,27/12/2024,27/12/2024,German,0,0,0,0,0,0
3,Al Baleed Resort Salalah by Anantara,NaN,5,Perfekt👍👍👍,"Die Hotelanlage,- das Essen wie auch das Perso...",0,COUPLES,31/12/2024,26/12/2024,25/12/2024,German,4,5,5,5,5,5
4,Al Baleed Resort Salalah by Anantara,Dubai,5,"Excellent for couples and families, very frien...",Definitely recommended to everyone. Asad and J...,0,NONE,31/12/2024,24/12/2024,23/12/2024,English,3,5,5,5,5,5
5,Al Baleed Resort Salalah by Anantara,Salzburg,5,REALLY highly recommended,Before starting saison (20th Dec.)... very cal...,0,COUPLES,31/12/2024,23/12/2024,23/12/2024,English,5,5,5,5,5,5
6,Al Baleed Resort Salalah by Anantara,Bacau,5,A wonderful place,"A quiet place, by the sea, with a gorgeous bea...",0,NONE,31/12/2024,23/12/2024,23/12/2024,English,4,5,5,5,5,5
7,Al Baleed Resort Salalah by Anantara,Bratislava,5,Beautiful & relaxed days in exclusive resort,We have spent beautiful & relaxed days in this...,0,COUPLES,31/12/2024,22/12/2024,21/12/2024,English,0,0,0,0,0,0
8,Al Baleed Resort Salalah by Anantara,NaN,5,"Very pleasant, you feel at home",An exceptional hotel and high quality staff \n...,0,COUPLES,31/12/2024,20/12/2024,20/12/2024,English,0,0,0,0,0,0
9,Al Baleed Resort Salalah by Anantara,NaN,5,top resort,"Superb location, accommodation and, above all,...",0,FRIENDS,31/12/2024,19/12/2024,19/12/2024,English,4,5,5,5,5,5
